Writing first Neural code in Pytorch<br>
1.Define the neural networks with some learnable parameters(wieghts)<br>
2.Iterate over the dataset of inputs<br>
3.Process input through the network<br>
4.Compute the loss<br>
5.Propogate gradients back into the network's parameter<br>
6.Update the weights of the network,typically using a simple update rule: weight = weight - learning_rate * gradient<br>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F


In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        #1 input image channel,6 output channels,5x5 square convolution
        #kernel
        self.conv1 = nn.Conv2d(1,6,5)
        self.conv2 = nn.Conv2d(6,16,5)
        # an affine operation:y = mx + b
        self.fc1 = nn.Linear(16 * 5 * 5,120)
        self.fc2 = nn.Linear(120,84)
        self.fc3 = nn.Linear(84,10)
        
    def forward(self,x):
        #Max pooling over a 2x2 window
        x = F.max_pool2d(F.relu(self.conv1(x)),(2,2))
        #if the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)),2)
        x = x.view(-1,self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    def num_flat_features(self,x):
        size = x.size()[1:] #all dimension except the batch dimension
        num_features = 1
        for s in size:
            num_features *=s
        return num_features
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [5]:
params = list(net.parameters())
print(len(params))
print(params[0].size())#Conv1's .weight

10
torch.Size([6, 1, 5, 5])


In [6]:
input = torch.randn(1,1,32,32)
out = net(input)
print(out)

tensor([[ 0.0381,  0.0124,  0.1099,  0.0305,  0.0637, -0.0175,  0.1510,
          0.0263, -0.0133,  0.0614]])


In [7]:
net.zero_grad()
out.backward(torch.randn(1,10))

In [8]:
output = net(input)
target = torch.arange(1,11) #a dummy target, for example
target = target.view(1,-1) #make is same shape as output
criterion = nn.MSELoss()

loss = criterion(output,target)
print(loss)

tensor(38.0066)


In [9]:
print(loss.grad_fn) #MSELoss
print(loss.grad_fn.next_functions[0][0]) #Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0]) #Relu


In [10]:
net.zero_grad() #zeros the gradient buffers of all parameters

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()
print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([ 0.,  0.,  0.,  0.,  0.,  0.])
conv1.bias.grad after backward
tensor(1.00000e-02 *
       [ 3.3559, -6.4005,  4.2183,  4.5251, -8.9534,  1.3757])


In [11]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

In [13]:
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(),lr = 0.01)
#in your training loop:
optimizer.zero_grad() #zero the gradient buffers
output = net(input)
loss = criterion(output,target)
loss.backward()
optimizer.step() #Does the update